# Fooooooooooooooocus
Colab NoteBook Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)

这是适用于 [Colab](https://colab.research.google.com) 部署 [Fooocus](https://github.com/lllyasviel/Fooocus) 的 Jupyter Notebook，使用时请按顺序执行 Jupyter Notebook 单元。

Colab 链接：<a href="https://colab.research.google.com/github/licyk/sd-webui-all-in-one/blob/main/fooocus_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 功能
1. 环境配置：配置安装的 PyTorch 版本、内网穿透的方式，并安装 [Fooocus](https://github.com/lllyasviel/Fooocus)，默认设置下已启用`配置环境完成后立即启动 Fooocus`选项，则安装完成后将直接启动 Fooocus，并显示访问地址。
2. 下载模型：下载可选列表中的模型（可选）。
3. 自定义模型下载：使用链接下载模型（可选）。
4. 启动 Fooocus：启动 Fooocus，并显示访问地址。


## 使用
1. 在 Colab -> 代码执行程序 > 更改运行时类型 -> 硬件加速器 选择`GPU T4`或者其他 GPU。
2. `环境配置`单元中的选项通常不需要修改，保持默认即可。
3. 运行`环境配置`单元，默认设置下已启用`配置环境完成后立即启动 Fooocus`选项，则环境配置完成后立即启动 Fooocus，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。
4. 如果未启用`配置环境完成后立即启动 Fooocus`选项，需要运行`启动 Fooocus`单元，此时将弹出 Google Drive 授权页面，根据提示进行操作。配置完成后将启动 Fooocus，Fooocus 的访问地址可在日志中查看。
5. 提示词查询工具：[SD 绘画提示词查询](https://licyk.github.io/t/tag)

## 提示
1. Colab 在关机后将会清除所有数据，所以每次重新启动时必须运行`环境配置`单元才能运行`启动 Fooocus`单元。
2. [Ngrok](https://ngrok.com) 内网穿透在使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
3. [Zrok](https://zrok.io) 内网穿透在使用前需要填写 Zrok Token, 可在 [Zrok](https://docs.zrok.io/docs/getting-started) 官网获取。
4. 每次启动 Colab 后必须运行`环境配置`单元，才能运行`启动 Fooocus`单元启动 Fooocus。
5. 其他功能有自定义模型下载等功能，根据自己的需求进行使用。
6. 运行`启动 Fooocus`时将弹出 Google Drive 访问授权提示，根据提示进行授权。授权后，使用 Fooocus 生成的图片将保存在 Google Drive 的`fooocus_output`文件夹中。
7. Gradio 的内网穿透地址可在启动 Fooocus 后日志中的`Running on public URL`查看。
8. Fooocus 启动预设文件可自行编写，并上传到 Github 或者其他平台并获取下载链接后，可以替换`Fooocus 启动预设文件地址`中的链接，进行环境配置时将使用该预设文件启动 Fooocus。预设文件可自行下载下来进行参考，其他有关说明可阅读：[Customization - lllyasviel/Fooocus](https://github.com/lllyasviel/Fooocus?tab=readme-ov-file#customization)。
9. 在`额外挂载目录设置`中可以挂载一些额外目录，如 LoRA 模型目录，挂载后的目录可在 Google Drive 的`fooocus_output`文件夹中查看和修改。可通过该功能在 Google Drive 上传模型并在 Fooocus 中使用。
10. 默认挂载以下目录到 Google Drive，可在 Google Drive 的`fooocus_output`文件夹中上传和修改文件：`outputs`, `presets`, `language`, `wildcards`, `config.txt`

In [ ]:
#@title 👇 环境配置
# Fooocus Manager 功能初始化部分, 通常不需要修改
# 如果需要查看完整代码实现, 可阅读: https://github.com/licyk/sd-webui-all-in-one/blob/main/sd_scripts_ipynb_core.py
#################################################################################################################
# SD_SCRIPTS_IPYNB_CORE_URL, FORCE_DOWNLOAD_CORE 参数可根据需求修改, 通常保持默认即可
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py" # Fooocus Manager 核心下载地址
FORCE_DOWNLOAD_CORE = False # 设置为 True 时, 即使 Fooocus Manager 已存在也会重新下载
#################################################################################################################
import os
import sys
import urllib.request
from pathlib import Path
try:
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / "sd_scripts_ipynb_core.py" # Fooocus Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[Manager] Fooocus Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[Manager] Fooocus Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"Fooocus Manager 核心模块下载错误: {e}")
os.environ["MANAGER_LOGGER_COLOR"] = "1"
os.environ["MANAGER_LOGGER_NAME"] = "Fooocus Manager"
from sd_scripts_ipynb_core import logger, VERSION, FooocusManager
logger.info("Fooocus Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")

#######################################################

#@markdown ## Fooocus 核心配置选项
#@markdown - Fooocus 分支仓库地址：
FOOOCUS_REPO = "https://github.com/lllyasviel/Fooocus" #@param {type:"string"}
#@markdown - Fooocus 启动预设文件地址：
FOOOCUS_PRESET = "https://github.com/licyk/sd-webui-all-in-one/raw/main/fooocus_config.json" #@param {type:"string"}
#@markdown - Fooocus 翻译文件下载地址：
FOOOCUS_TRANSLATION = "https://github.com/licyk/sd-webui-all-in-one/raw/main/fooocus_zh_cn.json" #@param {type:"string"}
#@markdown - Fooocus 依赖表名：
FOOOCUS_REQUIREMENTS = "requirements_versions.txt" #@param {type:"string"}
#@markdown - Fooocus 启动参数：
FOOOCUS_LAUNCH_ARGS = "--preset custom --language zh --async-cuda-allocation --disable-analytics --always-high-vram --always-download-new-model" #@param {type:"string"}
#@markdown ---
#@markdown ## PyTorch 组件版本选项：
#@markdown - PyTorch：
PYTORCH_VER = "torch==2.6.0+cu124 torchvision==0.21.0+cu124 torchaudio==2.6.0+cu124" #@param {type:"string"}
#@markdown - xFormers：
XFORMERS_VER = "xformers==0.0.29.post3" #@param {type:"string"}
#@markdown ---
#@markdown ## 包管理器选项：
#@markdown - 使用 uv 作为 Python 包管理器
USE_UV = True #@param {type:"boolean"}
#@markdown - PyPI 主镜像源
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" #@param {type:"string"}
#@markdown - PyPI 扩展镜像源
PIP_EXTRA_INDEX_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown - PyPI 额外镜像源
PIP_FIND_LINKS_MIRROR = "" #@param {type:"string"}
#@markdown - PyTorch 镜像源
PYTORCH_MIRROR = "https://download.pytorch.org/whl/cu124" #@param {type:"string"}
#@markdown ---
#@markdown ## 内网穿透选项：
#@markdown - 使用 remote.moe 内网穿透
USE_REMOTE_MOE = True #@param {type:"boolean"}
#@markdown - 使用 localhost.run 内网穿透
USE_LOCALHOST_RUN = True #@param {type:"boolean"}
#@markdown - 使用 pinggy.io 内网穿透
USE_PINGGY_IO = True #@param {type:"boolean"}
#@markdown - 使用 CloudFlare 内网穿透
USE_CLOUDFLARE = True #@param {type:"boolean"}
#@markdown - 使用 Gradio 内网穿透
USE_GRADIO = True #@param {type:"boolean"}
#@markdown - 使用 Ngrok 内网穿透（需填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取）
USE_NGROK = True #@param {type:"boolean"}
#@markdown - Ngrok Token
NGROK_TOKEN = ""  #@param {type:"string"}
#@markdown - 使用 Zrok 内网穿透（需填写 Zrok Token，可在 [Zrok](https://docs.zrok.io/docs/getting-started) 官网获取）
USE_ZROK = True #@param {type:"boolean"}
#@markdown - Zrok Token
ZROK_TOKEN = ""  #@param {type:"string"}
#@markdown ---
#@markdown ## 快速启动选项：
#@markdown - 配置环境完成后立即启动 Fooocus（并挂载 Google Drive）
QUICK_LAUNCH = True #@param {type:"boolean"}
#@markdown - 不重复配置环境（当重复运行环境配置时将不会再进行安装）
NO_REPEAT_CONFIGURE_ENV = True #@param {type:"boolean"}
#@markdown ---
#@markdown ## 其他选项：
#@markdown - 预下载模型时使用的下载器
MODEL_DOWNLOADER = "mix" #@param ["aria2", "request", "mix"]
#@markdown - 预下载模型时下载线程
DOWNLOAD_MODEL_THREAD = 16 #@param {type:"slider", min:1, max:64, step:1}
#@markdown - 清理无用日志
CLEAR_LOG = True #@param {type:"boolean"}
#@markdown - 检查可用 GPU
CHECK_AVALIABLE_GPU = True #@param {type:"boolean"}
#@markdown - 启用 TCMalloc 内存优化
ENABLE_TCMALLOC = True #@param {type:"boolean"}
#@markdown - 启用 CUDA Malloc 显存优化
ENABLE_CUDA_MALLOC = True #@param {type:"boolean"}

# @markdown ---
##############################################################################
# @markdown ## 额外挂载目录设置, 在启动时挂载目录到 Google Drive：
EXTRA_LINK_DIR = []

#@markdown - models/checkpoints 目录
link_checkpoints_dir = False # @param {type:"boolean"}
#@markdown - models/loras 目录
link_loras_dir = False # @param {type:"boolean"}
#@markdown - models/embeddings 目录
link_embeddings_dir = False # @param {type:"boolean"}
#@markdown - models/vae_approx 目录
link_vae_approx_dir = False # @param {type:"boolean"}
#@markdown - models/vae 目录
link_vae_dir = False # @param {type:"boolean"}
#@markdown - models/upscale_models 目录
link_upscale_models_dir = False # @param {type:"boolean"}
#@markdown - models/inpaint 目录
link_inpaint_dir = False # @param {type:"boolean"}
#@markdown - models/controlnet 目录
link_controlnet_dir = False # @param {type:"boolean"}
#@markdown - models/clip_vision 目录
link_clip_vision_dir = False # @param {type:"boolean"}
#@markdown - models/prompt_expansion 目录
link_prompt_expansion_dir = False # @param {type:"boolean"}
#@markdown - models/safety_checker 目录
link_safety_checker_dir = False # @param {type:"boolean"}
#@markdown - models/sam 目录
link_sam_dir = False # @param {type:"boolean"}


link_checkpoints_dir and EXTRA_LINK_DIR.append({"link_dir": "models/checkpoints"})
link_loras_dir and EXTRA_LINK_DIR.append({"link_dir": "models/loras"})
link_embeddings_dir and EXTRA_LINK_DIR.append({"link_dir": "models/embeddings"})
link_vae_approx_dir and EXTRA_LINK_DIR.append({"link_dir": "models/vae_approx"})
link_vae_dir and EXTRA_LINK_DIR.append({"link_dir": "models/vae"})
link_upscale_models_dir and EXTRA_LINK_DIR.append({"link_dir": "models/upscale_models"})
link_inpaint_dir and EXTRA_LINK_DIR.append({"link_dir": "models/inpaint"})
link_controlnet_dir and EXTRA_LINK_DIR.append({"link_dir": "models/controlnet"})
link_clip_vision_dir and EXTRA_LINK_DIR.append({"link_dir": "models/clip_vision"})
link_prompt_expansion_dir and EXTRA_LINK_DIR.append({"link_dir": "models/prompt_expansion"})
link_safety_checker_dir and EXTRA_LINK_DIR.append({"link_dir": "models/safety_checker"})
link_sam_dir and EXTRA_LINK_DIR.append({"link_dir": "models/sam"})

#######################################################

INSTALL_PARAMS = {
    "torch_ver": PYTORCH_VER or None,
    "xformers_ver": XFORMERS_VER or None,
    "use_uv": USE_UV,
    "pypi_index_mirror": PIP_INDEX_MIRROR or None,
    "pypi_extra_index_mirror": PIP_EXTRA_INDEX_MIRROR or None,
    "pypi_find_links_mirror": PIP_FIND_LINKS_MIRROR or None,
    # Colab 的环境暂不需要以下镜像源
    # "github_mirror": GITHUB_MIRROR or None,
    # "huggingface_mirror": HUGGINGFACE_MIRROR or None,
    "pytorch_mirror": PYTORCH_MIRROR or None,
    "fooocus_repo": FOOOCUS_REPO or None,
    "fooocus_requirements": FOOOCUS_REQUIREMENTS or None,
    "fooocus_preset": FOOOCUS_PRESET or None,
    "fooocus_translation": FOOOCUS_TRANSLATION or None,
    "model_downloader": MODEL_DOWNLOADER,
    "download_model_thread": DOWNLOAD_MODEL_THREAD,
    "check_avaliable_gpu": CHECK_AVALIABLE_GPU,
    "enable_tcmalloc": ENABLE_TCMALLOC,
    "enable_cuda_malloc": ENABLE_CUDA_MALLOC,
}

TUNNEL_PARAMS = {
    "use_ngrok": USE_NGROK,
    "ngrok_token": NGROK_TOKEN or None,
    "use_cloudflare": USE_CLOUDFLARE,
    "use_remote_moe": USE_REMOTE_MOE,
    "use_localhost_run": USE_LOCALHOST_RUN,
    "use_gradio": USE_GRADIO,
    "use_pinggy_io": USE_PINGGY_IO,
    "use_zrok": USE_ZROK,
    "zrok_token": ZROK_TOKEN or None,
    "message": "##### Fooocus 访问地址 #####",
}

FOOOCUS_PATH = "/content/Fooocus"
try:
    _ = fooocus_manager_has_init
except Exception:
    fooocus = FooocusManager(os.path.dirname(FOOOCUS_PATH), os.path.basename(FOOOCUS_PATH), port=7865)
    fooocus_manager_has_init = True

try:
    _ = fooocus_has_init # type: ignore
except Exception:
    fooocus_has_init = False

if NO_REPEAT_CONFIGURE_ENV:
    if not fooocus_has_init:
        fooocus.install(**INSTALL_PARAMS)
        INIT_CONFIG = 1
        fooocus_has_init = True
        CLEAR_LOG and fooocus.utils.clear_up()
        logger.info("Fooocus 运行环境配置完成")
    else:
        logger.info("检测到不重复配置环境已启用, 并且在当前 Colab 实例中已配置 Fooocus 运行环境, 不再重复配置 Fooocus 运行环境")
        logger.info("如需在当前 Colab 实例中重新配置 Fooocus 运行环境, 请在快速启动选项中取消不重复配置环境功能")
else:
    fooocus.install(**INSTALL_PARAMS)
    INIT_CONFIG = 1
    CLEAR_LOG and fooocus.utils.clear_up()
    logger.info("Fooocus 运行环境配置完成")

if QUICK_LAUNCH:
    logger.info("启动 Fooocus 中")
    os.chdir(FOOOCUS_PATH)
    fooocus.check_env(
        use_uv=USE_UV,
        requirements_file=FOOOCUS_REQUIREMENTS,
    )
    fooocus.mount_drive(EXTRA_LINK_DIR)
    fooocus.tun.start_tunnel(**TUNNEL_PARAMS)
    logger.info("Fooocus 启动参数: %s", FOOOCUS_LAUNCH_ARGS)
    if USE_GRADIO:
        !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS} --share
    else:
        !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS}
    CLEAR_LOG and fooocus.utils.clear_up()
    logger.info("已关闭 Fooocus")

In [ ]:
#@title 👇 下载模型（可选）

try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

# 模型下载
logger.info("下载模型中")
MODEL_LIST = []
#@markdown 选择下载的模型：
##############################

sd_xl_base_1_0_0_9vae = False # @param {type:"boolean"}
animapencilxl_v200 = False # @param {type:"boolean"}
bluepencilxl_v401 = False # @param {type:"boolean"}
anythingxl_xl = False # @param {type:"boolean"}
abyssorangexlelse_v10 = False # @param {type:"boolean"}
counterfeitxl_v1_0 = False # @param {type:"boolean"}
animeillustdiffusion_v061 = False # @param {type:"boolean"}
nekorayxl_v06w3 = False # @param {type:"boolean"}
animagine_xl_3_0 = False # @param {type:"boolean"}
animagine_xl_3_1 = False # @param {type:"boolean"}
animagine_xl_4_0 = False # @param {type:"boolean"}
heartofapplexl_v20 = False # @param {type:"boolean"}
heartofapplexl_v30 = False # @param {type:"boolean"}
holodayo_xl_2_1 = False # @param {type:"boolean"}
kivotos_xl_2_0 = False # @param {type:"boolean"}
sd_xl_anime_v52 = False # @param {type:"boolean"}
kohaku_xl_gamma_rev1 = False # @param {type:"boolean"}
kohakuxlbeta_beta7 = False # @param {type:"boolean"}
kohaku_xl_delta_rev1 = False # @param {type:"boolean"}
kohaku_xl_epsilon_rev1 = False # @param {type:"boolean"}
kohaku_xl_epsilon_rev2 = False # @param {type:"boolean"}
kohaku_xl_epsilon_rev3 = False # @param {type:"boolean"}
kohaku_xl_zeta = False # @param {type:"boolean"}
starryxlv52_v52 = False # @param {type:"boolean"}
baxlbartstylexlbluearchiveflatcelluloid_xlv1 = False # @param {type:"boolean"}
baxlbluearchiveflatcelluloidstyle_xlv3 = False # @param {type:"boolean"}
ponydiffusionv6xl_v6startwiththisone = False # @param {type:"boolean"}
pdforanime_v20 = False # @param {type:"boolean"}
tponynai3_v51weightoptimized = False # @param {type:"boolean"}
omegaponyxlanime_v20 = False # @param {type:"boolean"}
illustrious_xl_v0_1 = False # @param {type:"boolean"}
illustrious_xl_v1_0 = False # @param {type:"boolean"}
illustrious_xl_v1_1 = False # @param {type:"boolean"}
illustrious_xl_v2_0_stable = False # @param {type:"boolean"}
noobaixlnaixl_earlyaccessversion = False # @param {type:"boolean"}
noobaixlnaixl_epsilonpred05version = False # @param {type:"boolean"}
noobaixlnaixl_epsilonpred075 = False # @param {type:"boolean"}
noobaixlnaixl_epsilonpred077 = False # @param {type:"boolean"}
noobaixlnaixl_epsilonpred10version = False # @param {type:"boolean"}
noobaixlnaixl_epsilonpred11version = False # @param {type:"boolean"}
noobaixlnaixl_vpredtestversion = False # @param {type:"boolean"}
noobaixlnaixl_vpred05version = False # @param {type:"boolean"}
noobaixlnaixl_vpred06version = False # @param {type:"boolean"}
noobaixlnaixl_vpred065sversion = False # @param {type:"boolean"}
noobaixlnaixl_vpred075sversion = False # @param {type:"boolean"}
noobaixlnaixl_vpred09rversion = False # @param {type:"boolean"}
noobaixlnaixl_vpred10version = False # @param {type:"boolean"}


sd_xl_base_1_0_0_9vae and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors", "type": "checkpoints"})
animapencilxl_v200 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animaPencilXL_v200.safetensors", "type": "checkpoints"})
bluepencilxl_v401 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/bluePencilXL_v401.safetensors", "type": "checkpoints"})
anythingxl_xl and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors", "type": "checkpoints"})
abyssorangexlelse_v10 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors", "type": "checkpoints"})
counterfeitxl_v1_0 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors", "type": "checkpoints"})
animeillustdiffusion_v061 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors", "type": "checkpoints"})
nekorayxl_v06w3 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors", "type": "checkpoints"})
animagine_xl_3_0 and MODEL_LIST.append({"url": "https://huggingface.co/cagliostrolab/animagine-xl-3.0/resolve/main/animagine-xl-3.0.safetensors", "type": "checkpoints"})
animagine_xl_3_1 and MODEL_LIST.append({"url": "https://huggingface.co/cagliostrolab/animagine-xl-3.1/resolve/main/animagine-xl-3.1.safetensors", "type": "checkpoints"})
animagine_xl_4_0 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors", "type": "checkpoints"})
heartofapplexl_v20 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors", "type": "checkpoints"})
heartofapplexl_v30 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors", "type": "checkpoints"})
holodayo_xl_2_1 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors", "type": "checkpoints"})
kivotos_xl_2_0 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors", "type": "checkpoints"})
sd_xl_anime_v52 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors", "type": "checkpoints"})
kohaku_xl_gamma_rev1 and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-gamma/resolve/main/kohaku-xl-gamma-rev1.safetensors", "type": "checkpoints"})
kohakuxlbeta_beta7 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLBeta_beta7.safetensors", "type": "checkpoints"})
kohaku_xl_delta_rev1 and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-Delta/resolve/main/kohaku-xl-delta-rev1.safetensors", "type": "checkpoints"})
kohaku_xl_epsilon_rev1 and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon/resolve/main/kohaku-xl-epsilon-rev1.safetensors", "type": "checkpoints"})
kohaku_xl_epsilon_rev2 and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev2/resolve/main/kohaku-xl-epsilon-rev2.safetensors", "type": "checkpoints"})
kohaku_xl_epsilon_rev3 and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-Epsilon-rev3/resolve/main/kohaku-xl-epsilon-rev3.safetensors", "type": "checkpoints"})
kohaku_xl_zeta and MODEL_LIST.append({"url": "https://huggingface.co/KBlueLeaf/Kohaku-XL-Zeta/resolve/main/kohaku-xl-zeta.safetensors", "type": "checkpoints"})
starryxlv52_v52 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors", "type": "checkpoints"})
baxlbartstylexlbluearchiveflatcelluloid_xlv1 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBartstylexlBlueArchiveFlatCelluloid_xlv1.safetensors", "type": "checkpoints"})
baxlbluearchiveflatcelluloidstyle_xlv3 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors", "type": "checkpoints"})
ponydiffusionv6xl_v6startwiththisone and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors", "type": "checkpoints"})
pdforanime_v20 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors", "type": "checkpoints"})
tponynai3_v51weightoptimized and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors", "type": "checkpoints"})
omegaponyxlanime_v20 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors", "type": "checkpoints"})
illustrious_xl_v0_1 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors", "type": "checkpoints"})
illustrious_xl_v1_0 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors", "type": "checkpoints"})
illustrious_xl_v1_1 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.1.safetensors", "type": "checkpoints"})
illustrious_xl_v2_0_stable and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v2.0-stable.safetensors", "type": "checkpoints"})
noobaixlnaixl_earlyaccessversion and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors", "type": "checkpoints"})
noobaixlnaixl_epsilonpred05version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors", "type": "checkpoints"})
noobaixlnaixl_epsilonpred075 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors", "type": "checkpoints"})
noobaixlnaixl_epsilonpred077 and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors", "type": "checkpoints"})
noobaixlnaixl_epsilonpred10version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors", "type": "checkpoints"})
noobaixlnaixl_epsilonpred11version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpredtestversion and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred05version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred06version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred065sversion and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred075sversion and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred09rversion and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors", "type": "checkpoints"})
noobaixlnaixl_vpred10version and MODEL_LIST.append({"url": "https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors", "type": "checkpoints"})

##############################
fooocus.get_sd_model_from_list(MODEL_LIST)
CLEAR_LOG and fooocus.utils.clear_up()
logger.info("模型下载完成")

In [ ]:
#@title 👇 自定义模型下载
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

#@markdown ### 选择模型种类：
model_type = "loras" # @param ["checkpoints", "loras", "embeddings", "vae_approx", "vae", "upscale_models", "inpaint", "controlnet", "clip_vision", "prompt_expansion", "safety_checker", "sam"]
#@markdown ### 填写模型的下载链接：
model_url = "https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/CoolFlatColor.safetensors"  #@param {type:"string"}
#@markdown ### 填写模型的名称（包括后缀名）：
model_name = "CoolFlatColor.safetensors"  #@param {type:"string"}

fooocus.get_sd_model(
    url=model_url,
    filename=model_name or None,
    model_type=model_type,
)

In [ ]:
#@title 👇 启动 Fooocus
try:
    _ = INIT_CONFIG
except:
    raise Exception("未安装 Fooocus")

logger.info("启动 Fooocus")
os.chdir(FOOOCUS_PATH)
fooocus.check_env(
    use_uv=USE_UV,
    requirements_file=FOOOCUS_REQUIREMENTS,
)
fooocus.mount_drive(EXTRA_LINK_DIR)
fooocus.tun.start_tunnel(**TUNNEL_PARAMS)
logger.info("Fooocus 启动参数: %s", FOOOCUS_LAUNCH_ARGS)
if USE_GRADIO:
    !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS} --share
else:
    !python "{FOOOCUS_PATH}/launch.py" {FOOOCUS_LAUNCH_ARGS}
CLEAR_LOG and fooocus.utils.clear_up()
logger.info("已关闭 Fooocus")

In [ ]:
#@title 👇 文件下载工具

#@markdown ### 填写文件的下载链接：
url = ""  #@param {type:"string"}
#@markdown ### 填写文件的保存路径：
file_path = "/content"  #@param {type:"string"}
#@markdown ### 填写文件的保存名称 (可选)：
filename = ""  #@param {type:"string"}

fooocus.downloader.download_file(
    url=url,
    path=file_path or None,
    save_name=filename or None,
)
